# Biopython

![biopython logo](http://biopython.org/assets/images/biopython_logo_white.png)

## A quick overview
### [Guy Allard](mailto://w.g.allard@lumc.nl)

# What is Biopython?

## 'Python Tools for Computational Molecular Biology'

- Fully open-source
- Actively developed
- Large community

# What can it do?

Modules, classes and functions for manipulating biological data

- File parsers and writers.
  - Sequence files: fasta, fastq, genbank, abi, sff, etc.
  - Alignment files: clustal, emboss, phylip, nexus, etc.
  - Sequence search outputs: BLAST, HMMER, BLAT, etc.
  - Phylogenetic trees: newick, nexus, phyloxml, etc.
  - Sequence motifs: AlignAce, TRANSFAC, etc.
  - Others: PDB files, etc.
- Access to remote resources (e.g., Entrez, NCBI BLAST).
- Application wrappers.
- A simple graphing tool.
- Simple algorithms (e.g., pairwise alignment, cluster analysis).
- References such as codon tables and IUPAC sequences.

# Where can I find more information?

- [Biopython Homepage](http://biopython.org/)
- [Biopython development repository](http://github.com/biopython/biopython)
- [Biopython mailing list](http://lists.open-bio.org/pipermail/biopython/)
- [Biopython 'cookbook'](http://biopython.org/DIST/docs/tutorial/Tutorial.html) (essential reading!)

# Manipulating Sequence Data

## Bio.SeqIO

Input and output of sequence files.

- SeqIO.read
  - Read a file containing a single sequence
- SeqIO.parse 
  - Iterate over all sequences in a sequence file
- SeqIO.write
  - write sequences to a file

In [38]:
from Bio import SeqIO

# read the first sequence
for record in SeqIO.parse("../data/records.fa", "fasta"):
    dna = record
    break

print dna

ID: 1
Name: 1
Description: 1
Number of features: 0
Seq('TGGAACATGTCCCGCTAGCTTCTTCTTGCTAGCAGATTTTTTCAGTTGATCGTC...TCT', SingleLetterAlphabet())


Each record is an object with several fields, including:

- record.id
  - the sequence id
- record.name
  - sequence name, usually the same as the id
- record.description
  - sequence description

The actual sequence is a separate object contained within the record which can be accessed using record.seq

The sequence has an 'alphabet' associated with it which defines which letters are allowed.

Different alphabets are used for DNA, RNA, protein etc.

In [39]:
print dna.seq

TGGAACATGTCCCGCTAGCTTCTTCTTGCTAGCAGATTTTTTCAGTTGATCGTCACATGCGGTAGACTACCCAAGGTGTGACTACTCGCATGCCTGATCT


There are lots of built in methods that can be used to manipulate the sequence

The sequence acts like a string in many ways

In [40]:
# get the length of the sequence
print "length: ", len(dna.seq)

length:  100


In [41]:
# slice and dice
print dna.seq[:10]

TGGAACATGT


In [42]:
# change the case
print dna.seq.lower()

tggaacatgtcccgctagcttcttcttgctagcagattttttcagttgatcgtcacatgcggtagactacccaaggtgtgactactcgcatgcctgatct


In [43]:
# concatenate the first and last 10 nucleotides
print dna.seq[:10] + dna.seq[-10:]

TGGAACATGTTGCCTGATCT


But also has more sequence-specific methods

In [44]:
# complement
print dna.seq.complement()

ACCTTGTACAGGGCGATCGAAGAAGAACGATCGTCTAAAAAAGTCAACTAGCAGTGTACGCCATCTGATGGGTTCCACACTGATGAGCGTACGGACTAGA


In [45]:
# reverse complement
print dna.seq.reverse_complement()

AGATCAGGCATGCGAGTAGTCACACCTTGGGTAGTCTACCGCATGTGACGATCAACTGAAAAAATCTGCTAGCAAGAAGAAGCTAGCGGGACATGTTCCA


In [53]:
# transcribe from DNA to RNA
rna = dna.seq.transcribe()
print rna

UGGAACAUGUCCCGCUAGCUUCUUCUUGCUAGCAGAUUUUUUCAGUUGAUCGUCACAUGCGGUAGACUACCCAAGGUGUGACUACUCGCAUGCCUGAUCU


In [63]:
# Translate from nucleotide to protein
protein = dna.seq.translate()
print protein

WNMSR*LLLASRFFQLIVTCGRLPKV*LLACLI


Sequence records can easily be written to a file.

Specifying the file type allows conversion between different formats.

For example, to convert from a fastq file to fasta format:

In [66]:
records = SeqIO.parse("../data/easy.fastq", "fastq")
SeqIO.write(records, "tmp.fasta", "fasta")

1

# Remote files

NCBI allow for remote querying of their Entrez database, and Biopython allows us to use their services from within python.

We can use the Entrez.efetch utility to retrieve various records from one of NCBI's databases.

A full list of these services and their documentation can be found on the [Entrez utilities help page](https://www.ncbi.nlm.nih.gov/books/NBK25500/)

In [69]:
from Bio import Entrez

IMPORTANT:

To monitor potential excessive use of their services, NCBI requests you to specify your email address with each request.

With Biopython, you can set it once for your session like this:

In [70]:
Entrez.email = 'python@lumc.nl'

Now we can make a query of the database.

The Entrez.efetch function returns a file-like handle that instead of pointing to a local file, points to a remote resource.

In [77]:
efetch_handle = Entrez.efetch(db="nucleotide", id="NM_005804",
                              rettype="gb", retmode="text")

We can use the handle as if it were a normal file handle opened with ```open("filename", "r")```, and read from it using SeqIO.read()

In [78]:
ncbi_record = SeqIO.read(efetch_handle, 'genbank')

print ncbi_record

ID: NM_005804.3
Name: NM_005804
Description: Homo sapiens DExD-box helicase 39A (DDX39A), transcript variant 1, mRNA
Number of features: 25
/comment=REVIEWED REFSEQ: This record has been curated by NCBI staff. The
reference sequence was derived from DA432925.1, BC001009.2 and
BM792110.1.
This sequence is a reference standard in the RefSeqGene project.
On Oct 14, 2010 this sequence version replaced gi:21040370.
Summary: This gene encodes a member of the DEAD box protein family.
These proteins are characterized by the conserved motif
Asp-Glu-Ala-Asp (DEAD) and are putative RNA helicases. They are
implicated in a number of cellular processes involving alteration
of RNA secondary structure, such as translation initiation, nuclear
and mitochondrial splicing, and ribosome and spliceosome assembly.
Based on their distribution patterns, some members of the DEAD box
protein family are believed to be involved in embryogenesis,
spermatogenesis, and cellular growth and division. This gene is
thoug

It is also possible to query for multiple records

In [79]:
efetch_handle = Entrez.efetch(db="nucleotide", id=["NM_005804","NM_000967"],
                              rettype="gb", retmode="text")

Which can then be iterated over using ```SeqIO.parse```

In [81]:
for record in SeqIO.parse(efetch_handle, 'genbank'):
    print record.id, record.description

NM_005804.3 Homo sapiens DExD-box helicase 39A (DDX39A), transcript variant 1, mRNA
NM_000967.3 Homo sapiens ribosomal protein L3 (RPL3), transcript variant 1, mRNA


# Remote Tools

It is possible to use Biopyton with remote tools.

For example, we can submit a BLAST search to the NCBI service. ([Documentation here](https://www.ncbi.nlm.nih.gov/BLAST/Doc/urlapi.html))

We will use qblast function in the Bio.Blast.NCBIWWW module to perform a BLAST search using the record we retrieved earlier.

NOTE: It can take some time for the search results to become available

In [89]:
from Bio.Blast.NCBIWWW import qblast
blast_handle = qblast('blastn', 'refseq_mrna', ncbi_record.seq)

We can the read from the file handle using the ```Bio.SearchIO``` module.

In [90]:
from Bio import SearchIO
qresult = SearchIO.read(blast_handle, 'blast-xml')
print qresult

Program: blastn (2.7.0+)
  Query: No (1558)
         definition line
 Target: refseq_mrna
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|308522777|ref|NM_005804.3|  Homo sapiens DExD-box he...
            1      1  gi|1034056594|ref|XM_016946961.1|  PREDICTED: Pan trogl...
            2      1  gi|1034130164|ref|XM_016935303.1|  PREDICTED: Pan trogl...
            3      1  gi|675689963|ref|XM_003807080.2|  PREDICTED: Pan panisc...
            4      1  gi|1099186172|ref|XM_004060164.2|  PREDICTED: Gorilla g...
            5      1  gi|686757516|ref|XM_002828787.3|  PREDICTED: Pongo abel...
            6      1  gi|795239725|ref|XM_011953207.1|  PREDICTED: Colobus an...
            7      1  gi|1059109912|ref|XM_017851234.1|  PREDICTED: Rhinopith...
            8      1  gi|724815869|ref|XM_010361572.1|  PREDI

In [92]:
print qresult[0]

Query: No
       definition line
  Hit: gi|308522777|ref|NM_005804.3| (1558)
       Homo sapiens DExD-box helicase 39A (DDX39A), transcript variant 1, mRNA
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0    2810.93    1558         [0:1558]               [0:1558]


In [94]:
print qresult[1]

Query: No
       definition line
  Hit: gi|1034056594|ref|XM_016946961.1| (1530)
       PREDICTED: Pan troglodytes ATP-dependent RNA helicase DDX39A (LOC1079...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0    2695.52    1519         [0:1519]              [11:1530]


# That was just an overview

This lesson was just a small taste of what can be done with Biopython.

I strongly recommend looking at the [Biopython 'cookbook'](http://biopython.org/DIST/docs/tutorial/Tutorial.html) to get an idea of the wide range of things that you can do with it.

The lesson was based on previous material by [Wibowo Arindrarto](mailto://w.arindrarto@lumc.nl) and Martijn Vermaat.

License: [Creative Commons Attribution 3.0 License (CC-by)](http://creativecommons.org/licenses/by/3.0)